In [1]:
import sys
import pickle
import argparse
import os
import torch
import torchfile
import torchvision
import numpy as np
from torch import nn
from torch.autograd import Variable
from functools import reduce

In [2]:
class LBase(nn.Sequential):
    def __init__(self, fn, *args):
        super(LBase, self).__init__(*args)
        self.lambda_func = fn

    def forward_prepare(self, x):
        output = []

        for module in self._modules.values():
            output.append(module(x))

        return output if output else x


class LForward(LBase):
    def forward(self, x):
        return self.lambda_func(self.forward_prepare(x))


class LMap(LBase):
    def forward(self, x):
        return list(map(self.lambda_func, self.forward_prepare(x)))


class LReduce(LBase):
    def forward(self, x):
        return reduce(self.lambda_func, self.forward_prepare(x))

In [3]:
RS = nn.Sequential(
    nn.Conv2d(3, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.Sequential(
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                          nn.ReLU(),
                          nn.Conv2d(16, 16, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(16),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
    ),
    nn.Sequential(
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(16, 32, (3, 3), (2, 2), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 nn.Sequential(
                          nn.AvgPool2d((1, 1), (2, 2)),
                          LReduce(lambda x, y, dim=1: torch.cat((x, y), dim),
                                  LForward(lambda x: x),
                                  LForward(lambda x: x * 0),
                                  ),
                      ),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                          nn.ReLU(),
                          nn.Conv2d(32, 32, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(32),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
    ),
    nn.Sequential(
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(32, 64, (3, 3), (2, 2), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 nn.Sequential(
                          nn.AvgPool2d((1, 1), (2, 2)),
                          LReduce(lambda x, y, dim=1: torch.cat((x, y), dim),
                                  LForward(lambda x: x),
                                  LForward(lambda x: x * 0),
                                  ),
                      ),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
        nn.Sequential(
            LMap(lambda x: x,
                 nn.Sequential(
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                          nn.ReLU(),
                          nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1), 1, 1, bias=False),
                          nn.BatchNorm2d(64),
                      ),
                 LForward(lambda x: x),
                 ),
            LReduce(lambda x, y: x + y),
            nn.ReLU(),
        ),
    ),
    nn.AvgPool2d((8, 8), (1, 1)),
    LForward(lambda x: x.view(x.size(0), -1)),
    nn.Sequential(LForward(lambda x: x.view(1, -1) if 1 == len(x.size()) else x), nn.Linear(64, 10)),
)


In [4]:
GAMMA_F = 0.5
CHECK_STEP = 10000
D_PATH = 'MNIST.t7'
modelPath = 'm.pt'
GAMMA_T = 0.0001
L_RATE = 0.001
MAX_ITER = 10000
TR = False
PRINT_STEP = 1000
B_SIZE = 100
FILTERS = 15
ROUNDS = 5
CLASSES = 10

In [5]:
def _write(print_arr):
    for v in print_arr: sys.stdout.write(str(v) + '\t')
    sys.stdout.write('\n')
    sys.stdout.flush()


def _transform(X):
    tmp = np.zeros((np.shape(X)[0], 3, 38, 38))
    tmp[:, :, 2:34, 2:34] = X

    for i in range(np.shape(X)[0]):
        r1 = np.random.randint(4)
        r2 = np.random.randint(4)
        X[i] = tmp[i, :, r1: r1 + 32, r2: r2 + 32]

        if np.random.uniform() > .5:
            X[i] = X[i, :, :, ::-1]

    return X


def _performance(net, X, y, n):
    acc = 0.
    model.eval()
    out = np.zeros((X.shape[0], 10))

    for batch in range(int(X.shape[0] / B_SIZE)):
        start = batch * B_SIZE
        stop = (batch + 1) * B_SIZE - 1
        ints = np.linspace(start, stop, B_SIZE).astype(int)
        data = Variable(torch.from_numpy(X[ints])).float()

        for i in range(n):
            data = BLOCKS[i](data)

        output = net(data)
        acc += np.mean(torch.max(output, 1)[1].cpu().data.numpy() == y[ints])
        out[ints] = output.cpu().data.numpy()

    acc /= (X.shape[0] / B_SIZE)
    model.train()
    return acc, out

In [6]:
model = RS

In [7]:
BLOCKS = {0: nn.Sequential(model[0], model[1], model[2])}

for i in range(8):
    BLOCKS[1 + i] = model[3][i]

for i in range(8):
    BLOCKS[9 + i] = model[4][i]

for i in range(8):
    BLOCKS[17 + i] = model[5][i]

In [8]:
data = torchvision.datasets.SVHN("./svhn", download=True)

X_train = data.data[:60000]
y_train = data.labels[:60000]
X_test = data.data[60000:]
y_test = data.labels[60000:]

cut = int(np.shape(X_train)[0] / B_SIZE * B_SIZE)

train_size = cut
X_train = X_train[:cut]
y_train = y_train[:cut]

cut = int(np.shape(X_test)[0] / B_SIZE * B_SIZE)

test_size = cut
X_test = X_test[:test_size]
y_test = y_test[:test_size]

Using downloaded and verified file: ./svhn/train_32x32.mat


In [9]:
criterion = nn.CrossEntropyLoss()
a_prev = 0.0
a_curr = -1.0
s = np.zeros((train_size, CLASSES))
cost = np.zeros((train_size, CLASSES))
Xoutput_previous = np.zeros((train_size, CLASSES))
y_batch = np.zeros((B_SIZE))
y_batch_t = np.zeros((B_SIZE))
g_curr = 0
g_prev = GAMMA_F
iters = 0
tries = 0

for n in range(ROUNDS):
    gamma = -1
    Z = 0

    for i in range(train_size):
        l_sum = 0

        for l in range(CLASSES):
            if l != y_train[i]:
                cost[i][l] = np.exp(s[i][l] - s[i][int(y_train[i])])
                l_sum += cost[i][l]

        cost[i][int(y_train[i])] = -1 * l_sum
        Z += l_sum

    block = BLOCKS[n]
    ci = nn.Sequential(model[6], model[7], model[8])

    if n < 17:
        ci = nn.Sequential(BLOCKS[17], ci)

    if n < 9:
        ci = nn.Sequential(BLOCKS[9], ci)

    tmp_m = nn.Sequential(block, ci, nn.Softmax(dim=0))
    optimizer = torch.optim.Adam(tmp_m.parameters(), lr=L_RATE)
    tries = 0
    X_batch_t = torch.zeros(B_SIZE, FILTERS, 32, 32)

    while gamma < GAMMA_T and ((CHECK_STEP * tries) < MAX_ITER):
        train_acc = 0
        test_acc = 0
        err = 0

        for batch in range(1, CHECK_STEP + 1):
            optimizer.zero_grad()

            ints = np.random.random_integers(np.shape(X_train)[0] - 1, size=B_SIZE)
            X_batch = X_train[ints]
            y_batch = Variable(torch.from_numpy(y_train[ints])).long()

            if TR:
                X_batch = _transform(X_batch)

            data = Variable(torch.from_numpy(X_batch)).float()

            for i in range(n):
                data = BLOCKS[i](data)

            output = tmp_m(data)
            loss = torch.exp(criterion(output, y_batch))
            loss.backward()
            err += loss.data.item()

            output = tmp_m(data)
            train_acc += np.mean(torch.max(output, 1)[1].cpu().data.numpy() == y_train[ints])

            model.eval()
            ints = np.random.random_integers(np.shape(X_test)[0] - 1, size=(B_SIZE))
            X_batch = X_test[ints]
            data = Variable(torch.from_numpy(X_batch)).float()

            for i in range(n):
                data = BLOCKS[i](data)

            output = tmp_m(data)
            test_acc += np.mean(torch.max(output, 1)[1].cpu().data.numpy() == y_test[ints])
            model.train()

            if batch % PRINT_STEP == 0:
                train_acc /= PRINT_STEP
                test_acc /= PRINT_STEP
                err /= PRINT_STEP
                _write(['n', n, 'batch', iters + batch + (CHECK_STEP * tries), 'train_acc', train_acc, 'test_acc', test_acc])
                train_acc = 0
                test_acc = 0
                err = 0

            for p in tmp_m.parameters():
                p.grad.data.clamp_(-.1, .1)

            optimizer.step()

        train_acc, out = _performance(tmp_m, X_train, y_train, n)
        g_curr = -1 * np.sum(out * cost) / Z
        gamma = (g_curr ** 2 - g_prev ** 2) / (1 - g_prev ** 2)

        if gamma > 0:
            gamma = np.sqrt(gamma)
        else:
            gamma = -1 * np.sqrt(-1 * gamma)

        a_curr = 0.5 * np.log((1 + g_curr) / (1 - g_curr))

        tries += 1

        if gamma > GAMMA_T or ((CHECK_STEP * tries) >= MAX_ITER):
            iters = iters + (tries * CHECK_STEP)
            _write([gamma, g_curr, g_prev])
            _write(['a_{t+1}:', a_curr, 'gamma:', gamma])

    s += out * a_curr - Xoutput_previous * a_prev
    test_acc, _ = _performance(tmp_m, X_test, y_test, n)
    _write(['t', ROUNDS, 'n_batches:', tries * CHECK_STEP, 'test_acc:', test_acc])
    g_prev = g_curr

torch.save(model.state_dict(), modelPath)

/home/pikalm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: This function is deprecated. Please call randint(1, 59999 + 1) instead
/home/pikalm/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: DeprecationWarning: This function is deprecated. Please call randint(1, 13256 + 1) instead


n	0	batch	1000	train_acc	0.2601699999999999	test_acc	0.2522900000000005	
n	0	batch	2000	train_acc	0.47547999999999935	test_acc	0.45938000000000007	
n	0	batch	3000	train_acc	0.5665199999999996	test_acc	0.5540099999999998	
n	0	batch	4000	train_acc	0.642590000000001	test_acc	0.6291400000000011	
n	0	batch	5000	train_acc	0.7017799999999998	test_acc	0.6887399999999999	
n	0	batch	6000	train_acc	0.7331199999999995	test_acc	0.717440000000001	
n	0	batch	7000	train_acc	0.7554899999999998	test_acc	0.7378699999999992	
n	0	batch	8000	train_acc	0.7777499999999989	test_acc	0.7648499999999985	
n	0	batch	9000	train_acc	0.7918999999999977	test_acc	0.7762799999999985	
n	0	batch	10000	train_acc	0.7984799999999992	test_acc	0.7872499999999986	
-0.5659286583433925	0.09896244363007538	0.5	
a_{t+1}:	0.09928742043677384	gamma:	-0.5659286583433925	
t	5	n_batches:	10000	test_acc:	0.8076487893188502	
n	1	batch	11000	train_acc	0.8022599999999999	test_acc	0.7973499999999987	
n	1	batch	12000	train_acc	0.82577000000000